In [4]:
import numpy as np
from utils import get_data, accuracy, save_Yte
from kernels import GaussianKernel, LinearKernel
from tqdm import tqdm
from scipy import optimize

Xtr, Xte, Ytr = get_data()

In [5]:
Xtr = Xtr[:,:1024:512]
Xte = Xte[:,:1024:512]

In [3]:
Xtr.shape

(5000, 2)

In [2]:
class KSVC:
    
    def __init__(self, kernel, C, class_id, epsilon, verbose):
        self.kernel = kernel        
        self.C = C                               
        self.alpha = None
        self.support = None
        self.epsilon = epsilon
        self.norm_f = None 
        self.verbose = verbose 
        self.class_id = class_id 
       
    def fit(self, X, y, K):
        if self.verbose: print("Fitting...")
        N = len(y)
        # K = self.kernel(X,X)
        self.X = X
        # self.y = y
        self.y = (y == self.class_id)

        # Lagrange dual problem
        def loss(alpha):
            return 1/2 * alpha.T @ np.diag(y) @ K @ np.diag(y) @ alpha - alpha.sum()
        # Partial derivate of Ld on alpha
        def grad_loss(alpha):
            return np.diag(y) @ K @ np.diag(y) @ alpha - 1

        fun_eq = lambda alpha: - alpha.T @ y
        jac_eq = lambda alpha: - y
        fun_ineq = lambda alpha: self.C - alpha
        jac_ineq = lambda alpha: - np.eye(N)
        fun_ineq_pos = lambda alpha: alpha
        jac_ineq_pos = lambda alpha: np.eye(N)
        
        constraints = ({'type': 'eq',  'fun': fun_eq, 'jac': jac_eq},
                       {'type': 'ineq', 'fun': fun_ineq , 'jac': jac_ineq},
                       {'type': 'ineq', 'fun': fun_ineq_pos, 'jac': jac_ineq_pos})

        optRes = optimize.minimize(fun=lambda alpha: loss(alpha),
                                   x0=np.ones(N), 
                                   method='SLSQP', 
                                   jac=lambda alpha: grad_loss(alpha), 
                                   constraints=constraints)
        self.alpha = optRes.x

        self.supportIndices = np.where((self.alpha > self.epsilon) & (self.alpha < self.C*np.ones(N) - self.epsilon))
        self.support = X[self.supportIndices]
        self.b = (self.y - self.alpha.T @ np.diag(self.y) @ K)[self.supportIndices].mean()
        self.norm_f = self.alpha.T @ np.diag(self.y) @ K @ np.diag(self.y) @ self.alpha

    def predict(self, K):
        if self.verbose: print("Predicting...")
        d = self.alpha.T @ np.diag(self.y) @ K
        return d + self.b

class MultiKSVC:

    def __init__(self, kernel, C, epsilon=1e-3, verbose=False):
        self.C = C        
        self.kernel = kernel        
        self.epsilon = epsilon        
        self.verbose = verbose
        self.KSVC_list = [KSVC(self.kernel, self.C, self.epsilon, class_id, self.verbose) for class_id in range(10)]
    
    def fit(self, X, y):
        self.X = X
        K = self.kernel(X,X)
        for ksvc in tqdm(self.KSVC_list, desc="Fitting"):
            ksvc.fit(X, y, K)
    
    def predict(self, x):
        K = self.kernel(self.X, x)
        pred = np.array([ksvc.predict(K) for ksvc in self.KSVC_list]).T
        pred = np.argmax(pred, axis=1)
        return pred

In [3]:
ksvc = MultiKSVC(kernel=GaussianKernel(sigma=1.5).kernel, C=100.)
ksvc.fit(Xtr, Ytr)
pred_test = ksvc.predict(Xte)
pred_train = ksvc.predict(Xtr)

Fitting:   0%|          | 0/10 [00:08<?, ?it/s]


KeyboardInterrupt: 